In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2525381/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_7_9_6,0.998798,0.955272,0.998987,0.997403,0.001330,0.003319,0.001504,0.002465,"Hidden Size=[25, 15], regularizer=0.01, learni..."
1,model_7_9_7,0.998793,0.947515,0.998876,0.997000,0.001336,0.003894,0.001668,0.002847,"Hidden Size=[25, 15], regularizer=0.01, learni..."
2,model_7_9_5,0.998781,0.963164,0.999099,0.997812,0.001349,0.002733,0.001338,0.002077,"Hidden Size=[25, 15], regularizer=0.01, learni..."
3,model_7_9_8,0.998772,0.940003,0.998769,0.996610,0.001359,0.004452,0.001828,0.003217,"Hidden Size=[25, 15], regularizer=0.01, learni..."
4,model_7_9_9,0.998741,0.932813,0.998664,0.996235,0.001394,0.004985,0.001982,0.003572,"Hidden Size=[25, 15], regularizer=0.01, learni..."
...,...,...,...,...,...,...,...,...,...,...
2868,model_7_3_12,0.946060,0.884647,0.965976,0.930386,0.059696,0.195345,0.083005,0.142479,"Hidden Size=[25, 15], regularizer=0.01, learni..."
2888,model_19_2_2,0.945167,0.950288,0.942765,0.951461,0.060684,0.048886,0.094978,0.070576,"Hidden Size=[35, 15], regularizer=0.01, learni..."
2898,model_9_8_1,0.944743,0.936145,0.992734,0.941669,0.061154,0.092128,0.001258,0.049366,"Hidden Size=[25, 5], regularizer=0.01, learnin..."
2904,model_7_3_11,0.944553,0.890414,0.967772,0.933920,0.061363,0.185578,0.078622,0.135246,"Hidden Size=[25, 15], regularizer=0.01, learni..."
